# Configuração do ambiente e carregamento do catálogo

**Lembre-se de baixar o arquivo json que garante acesso ao google storage**
 - monitor-rosa-escrita.json: acesso para criação de tabelas;
 - monitor-rosa-leitura.json: acesso para analises e consultas.

**Se o seu usuário possui acesso ao drive compartilhado, remova os comentários a seguir**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Escolha do ambiente
São disponibilizadas duas opções de execução (dependendo do arquivo de credencial utilizado)
 - gcp-leitura.json: disponível para analises e usuários que não possuem permissão de escrita;
 - gcp-escrita.json: disponível para atualização de bases (testes de desenvolvimento ou ambiente de produção).

In [2]:
import os
if os.path.isfile('/content/monitor-rosa-leitura.json'):
    datalake_mode = 'leitura'
    %env SERVICE_ACCOUNT_USER=acesso-leitura@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-leitura.json
elif os.path.isfile('/content/monitor-rosa-escrita.json'):
    datalake_mode = 'escrita'
    %env SERVICE_ACCOUNT_USER=acesso-escrita@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-escrita.json
else:
    assert(os.path.isdir('/content/drive/Shareddrives/monitor-rosa-gold') == True)
    datalake_mode = 'shared_drive'
    %env SERVICE_ACCOUNT_USER=''
    %env SERVICE_ACCOUNT_JSON=''
datalake_mode

env: SERVICE_ACCOUNT_USER=''
env: SERVICE_ACCOUNT_JSON=''


'shared_drive'

In [3]:
!rm -r sus-kpis-analysis
!git clone https://github.com/heber-augusto/sus-kpis-analysis.git

rm: cannot remove 'sus-kpis-analysis': No such file or directory
Cloning into 'sus-kpis-analysis'...
remote: Enumerating objects: 2081, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 2081 (delta 63), reused 15 (delta 15), pack-reused 1972 (from 2)
Receiving objects: 100% (2081/2081), 4.90 MiB | 7.23 MiB/s, done.
Resolving deltas: 100% (1022/1022), done.


## Instalação de libs Python, inicialização de variáveis de ambiente e configuração/instalação do Spark

In [4]:
!pip install -r /content/sus-kpis-analysis/sia/etls/requirements.txt

%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.5.5-bin-hadoop3
%env SPARK_VERSION=3.5.5

!source /content/sus-kpis-analysis/sia/etls/bin/setup_spark_env.sh '/content/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 20.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747862 sha256=ac510da4fd1ab94052fc7375c21e07628f84ec1bcef7b674644bd916212a4644
  Stored in directory: /root/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.5.5-bin-hadoop3
env: SPARK_VERSION=3.5.5
starting spark env setup 
installing and downloading packages
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does no

In [5]:
if datalake_mode != 'shared_drive':
    %env XDG_CONFIG_HOME=/content/datalake
    !source /content/sus-kpis-analysis/sia/etls/bin/install-google-drive-ocamlfuse.sh
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ABIY-a4qrdY9Uk9PVA' 'monitor-rosa-bronze' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ALl0owLNr53oUk9PVA' 'monitor-rosa-silver' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0AMHp9pBeLvZiUk9PVA' 'monitor-rosa-gold' $SERVICE_ACCOUNT_JSON '/content'


## Inicializa variáveis de acesso ao delta lake criado no google storage

- Local do arquivo de credencial do storage, diretorio do warehouse e path do spark:

> O arquivo json_file_name deve ser enviado para o ambiente e deve ser utilizado um com as devidas permissões (em caso de escrita)

> O caminho do warehouse pode ser alterado em caso de testes de escritas locais.

> O caminho do spark é setado pelo script de configuração

In [6]:
import os

lake_prefix = "temp-output"

if datalake_mode in ('leitura','shared_drive',):
    warehouse_dir = f"/content/datalake/{lake_prefix}/"

if datalake_mode == 'escrita':
    warehouse_dir = f"/content/datalake/"

spark_path = os.getenv('SPARK_HOME')
spark_path

'/content/spark-3.5.5-bin-hadoop3'

## Inclusão da pasta do repositório no python path

Procedimento permite que funções e classes presentes no repositório sejam utilizadas

In [7]:
import sys
sys.path.append('/content/sus-kpis-analysis')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor',
 '/root/.ipython',
 '/content/sus-kpis-analysis']

## Importação de funções utilizadas pelo código

In [8]:
from sia.etls.lib.catalog_loader import DeltaLakeDatabaseFsCreator, load_entire_catalog_fs, load_entire_catalog_fs_v2
from sia.etls.lib.table_utilities import vacuum_tables_from_database, table_exists
from sia.etls.lib.fs_spark_session import create_fs_spark_session
from sia.etls.lib.bronze_files_utilities import get_pending_files_from_bronze
from sia.etls.lib.delta_table_creators import ParquetToDelta

## Cria Sessão Spark conectada ao Delta Lake presente no Google Storage

In [9]:
spark = create_fs_spark_session(
    warehouse_dir=warehouse_dir,
    spark_path=spark_path
)


## Refresh do catálogo para utilizar consultas

In [10]:
zone_names = ['monitor-rosa-bronze', 'monitor-rosa-silver', 'monitor-rosa-gold']
zone_names = ['monitor-rosa-silver', 'monitor-rosa-gold']
if datalake_mode in ('leitura', 'escrita'):
    zone_paths = [f'/content/datalake/{zone_name}/databases' for zone_name in zone_names]
else:
    zone_paths = [f'/content/drive/Shareddrives/{zone_name}/databases' for zone_name in zone_names]


# carrega catalogo de banco de dados, na zona bronze
database_filter = None #['cnes_bronze.db',]
table_filter = None #['cnes_bronze.sr',]

for databases_path in zone_paths:
    load_entire_catalog_fs_v2(
        spark_session = spark,
        databases_path = databases_path,
        use_db_folder_path=(datalake_mode == 'escrita'),
        database_filter=database_filter,
        table_filter=table_filter
    )

['cancer_data.db', 'ibge_silver.db', 'cancer_mama_silver.db']
Banco de dados cancer_data criado.
listando conteúdos do caminho /content/drive/Shareddrives/monitor-rosa-silver/databases e database cancer_data
prefix: /content/drive/Shareddrives/monitor-rosa-silver/databases/cancer_data.db/
table_list: ['aq_filtered', 'ar_filtered', 'dados_estados_mensal', 'dados_municipios_mensal', 'pacientes', 'procedimentos', 'procedimentos_e_pacientes', 'demografia_municipios', 'cadastro_municipios']
Tabela aq_filtered criada
Tabela aq_filtered criada com comando CREATE TABLE IF NOT EXISTS cancer_data.aq_filtered USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-silver/databases/cancer_data.db/aq_filtered'
Tabela ar_filtered criada
Tabela ar_filtered criada com comando CREATE TABLE IF NOT EXISTS cancer_data.ar_filtered USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-silver/databases/cancer_data.db/ar_filtered'
Tabela dados_estados_mensal criada
Tabela dados_estados_mensal 

# Exemplos de consultas

## Listagem de bancos e tabelas

In [11]:
databases = spark.sql(f"SHOW DATABASES;")
databases.show()

+------------------+
|         namespace|
+------------------+
|       cancer_data|
|       cancer_mama|
|cancer_mama_silver|
|           default|
|       ibge_silver|
+------------------+



In [ ]:
for row in databases.collect():
    spark.sql(f"SHOW TABLES FROM {row['namespace']};").show(truncate=False)

+-----------+-------------------------+-----------+
|namespace  |tableName                |isTemporary|
+-----------+-------------------------+-----------+
|cancer_data|aq_filtered              |false      |
|cancer_data|ar_filtered              |false      |
|cancer_data|cadastro_municipios      |false      |
|cancer_data|dados_estados_mensal     |false      |
|cancer_data|dados_municipios_mensal  |false      |
|cancer_data|demografia_municipios    |false      |
|cancer_data|pacientes                |false      |
|cancer_data|procedimentos            |false      |
|cancer_data|procedimentos_e_pacientes|false      |
+-----------+-------------------------+-----------+

+-----------+-------------------------+-----------+
|namespace  |tableName                |isTemporary|
+-----------+-------------------------+-----------+
|cancer_mama|dados_estados_mensal     |false      |
|cancer_mama|dados_municipios_mensal  |false      |
|cancer_mama|pacientes                |false      |
|cancer_mam

In [12]:
dados_estad_mensal_df = spark.sql(f"""
    SELECT
        year(to_date(data, 'yyyyMM')) AS ano
        ,primeiro_estadiamento
        --SUM(custo) AS custo_estadiamento
        ,COUNT(DISTINCT(paciente)) AS numero_pacientes
        --,SUM(DISTINCT(obito)) AS obitos
        --,SUM(DISTINCT(indicacao_obito)) AS obito_futuro
        --,COUNT(1) AS numero_procedimentos
    FROM
        (SELECT * FROM cancer_mama.procedimentos_e_pacientes ORDER BY data)
    GROUP BY ano, primeiro_estadiamento
""")


In [13]:
df = dados_estad_mensal_df.toPandas()

In [14]:
# prompt: Usando o DataFrame df: stacked percentage of numero_pacientes by ano and estadiamento, using 0 in the bottom, and 1, 2, 3, 4 ordered to the top

import pandas as pd
import altair as alt

# Create a DataFrame (replace with your actual data)
# Assuming your DataFrame is named 'df'

# Convert 'primeiro_estadiamento' to numeric, handling errors
df['primeiro_estadiamento'] = pd.to_numeric(df['primeiro_estadiamento'], errors='coerce')

# Group by 'ano' and 'primeiro_estadiamento'
grouped_data = df.groupby(['ano', 'primeiro_estadiamento'])['numero_pacientes'].sum().reset_index()

# Calculate the total number of patients for each year
total_patients = grouped_data.groupby('ano')['numero_pacientes'].sum()

# Calculate the percentage for each group within each year
grouped_data['percentage'] = grouped_data.apply(lambda x: (x['numero_pacientes'] / total_patients[x['ano']]) * 100, axis=1)

# Create the stacked percentage chart using Altair
chart = alt.Chart(grouped_data).mark_bar().encode(
    x='ano:N',  # Categorical x-axis for years
    y=alt.Y('percentage:Q', stack='normalize', axis=alt.Axis(format='%')), # Stacked percentage with percentage formatting
    color=alt.Color('primeiro_estadiamento:N', sort='ascending'),  # Color by stage
    order=alt.Order('primeiro_estadiamento:N', sort='ascending')  # Order bars by stage
).properties(
    title='Stacked Percentage of Patients by Year and Stage'
)
chart


alt.Chart(...)

In [15]:
# prompt: # prompt: Usando o DataFrame df crie uma tabela com o percentual de numero_pacientes por ano e estadiamento, coloque os valores percentuais em colunas, seguindo a sequencia 0, 1, 2 , 3 e 4. Cada ano deve ser uma linha.

import pandas as pd
# Pivot the table to have stages as columns and years as rows
pivot_df = pd.pivot_table(grouped_data, values='percentage', index='ano', columns='primeiro_estadiamento', aggfunc='sum', fill_value=0)

# Rename columns for clarity (optional)
pivot_df = pivot_df.rename(columns={0: '0%', 1: '1%', 2: '2%', 3: '3%', 4: '4%'})

# Display the resulting table
pivot_df


primeiro_estadiamento,0%,1%,2%,3%,4%
ano,,,,,
2008,3.814595,19.434676,38.610637,27.401286,10.738806
2009,3.781282,19.808988,37.901885,28.969949,9.537896
2010,3.798612,20.187747,37.050392,30.151011,8.812237
2011,3.731716,20.767749,36.435248,30.770603,8.294684
2012,3.956734,21.276327,36.003692,30.697060,8.066187
2013,4.016375,21.590200,35.432840,31.020818,7.939767
2014,3.613773,21.706081,35.280293,31.275604,8.124249
2015,3.455347,21.380175,34.668415,32.082392,8.413671
2016,3.519766,21.235318,34.211197,32.369156,8.664563


In [16]:
spark.sql(f"""
  SELECT
      COUNT(DISTINCT(paciente)) AS numero_pacientes, count(1)
  FROM cancer_mama.pacientes AS c
  where data_primeiro_estadiamento = '202501'
""").show()

+----------------+--------+
|numero_pacientes|count(1)|
+----------------+--------+
|            4076|    4076|
+----------------+--------+



In [17]:
spark.sql(f"""
  SELECT
      COUNT(DISTINCT(paciente)) AS numero_pacientes, count(1)
  FROM cancer_mama.procedimentos AS c
  where data = '202501'
""").show()

+----------------+--------+
|numero_pacientes|count(1)|
+----------------+--------+
|          154351|  166837|
+----------------+--------+



In [18]:
spark.sql(f"""
    SELECT
        COUNT(DISTINCT(paciente)) AS numero_pacientes
    FROM
        cancer_mama.procedimentos_e_pacientes
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          154351|
+----------------+



In [19]:

spark.sql(f"""
  SELECT
      COUNT(DISTINCT(c.paciente)) AS numero_pacientes, count(1)
  FROM cancer_mama.procedimentos AS c
  LEFT JOIN cancer_mama.pacientes AS p
  ON c.paciente = p.paciente
  where data = '202501'
""").show()

+----------------+--------+
|numero_pacientes|count(1)|
+----------------+--------+
|          154351|  166837|
+----------------+--------+



In [20]:
spark.sql(f"""
  SELECT
      COUNT(DISTINCT(c.paciente)) AS numero_pacientes, count(1)
  FROM cancer_mama.procedimentos AS c
  full outer JOIN cancer_mama.pacientes AS p
  ON c.paciente = p.paciente
  where data = '202501'
""").show()

+----------------+--------+
|numero_pacientes|count(1)|
+----------------+--------+
|          154351|  166837|
+----------------+--------+



In [21]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes
    FROM
        cancer_mama.dados_estados_mensal
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          154871|
+----------------+



In [22]:
spark.sql(f"""
    SELECT
        count(distinct(pp.paciente)) AS numero_pacientes, cadastro_cidades.nome_uf
    FROM
        cancer_mama.procedimentos_e_pacientes pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501'
    group by nome_uf
    order by 1

""").show()

+----------------+-------------------+
|numero_pacientes|            nome_uf|
+----------------+-------------------+
|               5|               NULL|
|              26|              Amapá|
|             297|            Roraima|
|             425|               Acre|
|            1105|          Tocantins|
|            1370|        Mato Grosso|
|            1403|           Rondônia|
|            1608|            Paraíba|
|            1615|            Sergipe|
|            1649|              Goiás|
|            1826|           Amazonas|
|            2003|   Distrito Federal|
|            2268|               Pará|
|            2380|              Piauí|
|            2499|            Alagoas|
|            2504| Mato Grosso do Sul|
|            3345|           Maranhão|
|            3450|Rio Grande do Norte|
|            4087|     Espírito Santo|
|            4912|         Pernambuco|
+----------------+-------------------+
only showing top 20 rows



In [23]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes, estado
    FROM
        cancer_mama.dados_estados_mensal
    where data = '202501'
    group by estado
    order by 1

""").show()

+----------------+-------------------+
|numero_pacientes|             estado|
+----------------+-------------------+
|               5|               NULL|
|              26|              Amapá|
|             297|            Roraima|
|             425|               Acre|
|            1106|          Tocantins|
|            1370|        Mato Grosso|
|            1403|           Rondônia|
|            1608|            Paraíba|
|            1618|            Sergipe|
|            1653|              Goiás|
|            1826|           Amazonas|
|            2003|   Distrito Federal|
|            2296|               Pará|
|            2397|              Piauí|
|            2500|            Alagoas|
|            2505| Mato Grosso do Sul|
|            3355|           Maranhão|
|            3501|Rio Grande do Norte|
|            4089|     Espírito Santo|
|            4912|         Pernambuco|
+----------------+-------------------+
only showing top 20 rows



In [ ]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes, cadastro_cidades.nome_uf as estado
    FROM
        cancer_mama.dados_municipios_mensal mm
        LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
        ON int(mm.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501'
    group by estado
    order by 1

""").show()

+----------------+-------------------+
|numero_pacientes|             estado|
+----------------+-------------------+
|               5|               NULL|
|              26|              Amapá|
|             297|            Roraima|
|             425|               Acre|
|            1106|          Tocantins|
|            1370|        Mato Grosso|
|            1403|           Rondônia|
|            1608|            Paraíba|
|            1618|            Sergipe|
|            1653|              Goiás|
|            1826|           Amazonas|
|            2003|   Distrito Federal|
|            2296|               Pará|
|            2397|              Piauí|
|            2500|            Alagoas|
|            2505| Mato Grosso do Sul|
|            3355|           Maranhão|
|            3501|Rio Grande do Norte|
|            4089|     Espírito Santo|
|            4912|         Pernambuco|
+----------------+-------------------+
only showing top 20 rows



In [24]:
spark.sql(f"""
    SELECT a.estado, b.nome_uf, a.numero_pacientes, b.numero_pacientes_b, a.numero_pacientes - b.numero_pacientes_b as diferenca
    FROM

    (SELECT
        SUM(numero_pacientes) AS numero_pacientes, cadastro_cidades.nome_uf as estado
    FROM
        cancer_mama.dados_municipios_mensal mm
        LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
        ON int(mm.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501'
    group by estado
    order by 1) A

    LEFT JOIN

    (    SELECT
        count(distinct(pp.paciente)) AS numero_pacientes_b, cadastro_cidades.nome_uf
    FROM
        cancer_mama.procedimentos_e_pacientes pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501'
    group by nome_uf
    order by 1) B

    ON A.estado = B.nome_uf
    where a.numero_pacientes != b.numero_pacientes_b
    order by diferenca

""").show()

+-------------------+-------------------+----------------+------------------+---------+
|             estado|            nome_uf|numero_pacientes|numero_pacientes_b|diferenca|
+-------------------+-------------------+----------------+------------------+---------+
|            Alagoas|            Alagoas|            2500|              2499|        1|
|          Tocantins|          Tocantins|            1106|              1105|        1|
| Mato Grosso do Sul| Mato Grosso do Sul|            2505|              2504|        1|
|     Espírito Santo|     Espírito Santo|            4089|              4087|        2|
|     Santa Catarina|     Santa Catarina|            9224|              9221|        3|
|            Sergipe|            Sergipe|            1618|              1615|        3|
|              Goiás|              Goiás|            1653|              1649|        4|
|           Maranhão|           Maranhão|            3355|              3345|       10|
|              Piauí|           

In [25]:
spark.sql(f"""
    SELECT
        count(distinct(pp.paciente)) AS numero_pacientes, cadastro_cidades.nome_uf
    FROM
        cancer_mama.procedimentos_e_pacientes pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501'  and cadastro_cidades.nome_uf = 'Minas Gerais'
    group by nome_uf
    order by 1

""").show()

+----------------+------------+
|numero_pacientes|     nome_uf|
+----------------+------------+
|           16436|Minas Gerais|
+----------------+------------+



In [26]:
spark.sql(f"""
    SELECT
        count(distinct(pp.paciente)) AS numero_pacientes, cadastro_cidades.nome_uf, cadastro_cidades.id as cidade
    FROM
        cancer_mama.procedimentos_e_pacientes pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'
    group by nome_uf, cidade
    order by 3

""").show()

+----------------+------------+-------+
|numero_pacientes|     nome_uf| cidade|
+----------------+------------+-------+
|               7|Minas Gerais|3100104|
|              23|Minas Gerais|3100203|
|               2|Minas Gerais|3100302|
|               6|Minas Gerais|3100401|
|               6|Minas Gerais|3100500|
|               3|Minas Gerais|3100609|
|               1|Minas Gerais|3100708|
|               4|Minas Gerais|3100807|
|               9|Minas Gerais|3100906|
|               6|Minas Gerais|3101003|
|              13|Minas Gerais|3101102|
|               5|Minas Gerais|3101201|
|               6|Minas Gerais|3101300|
|               4|Minas Gerais|3101409|
|               3|Minas Gerais|3101508|
|               4|Minas Gerais|3101607|
|               2|Minas Gerais|3101631|
|              19|Minas Gerais|3101706|
|               1|Minas Gerais|3101805|
|              25|Minas Gerais|3101904|
+----------------+------------+-------+
only showing top 20 rows



In [27]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes, mm.municipio as cidade
    FROM
        cancer_mama.dados_municipios_mensal mm

    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(mm.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'

    group by cidade
    order by 2

""").show()

+----------------+------+
|numero_pacientes|cidade|
+----------------+------+
|               7|310010|
|              23|310020|
|               2|310030|
|               6|310040|
|               6|310050|
|               3|310060|
|               1|310070|
|               4|310080|
|               9|310090|
|               6|310100|
|              13|310110|
|               5|310120|
|               6|310130|
|               4|310140|
|               3|310150|
|               4|310160|
|               2|310163|
|              19|310170|
|               1|310180|
|              25|310190|
+----------------+------+
only showing top 20 rows



In [50]:
spark.sql(f"""

    SELECT sum(A.numero_pacientes), sum(B.numero_pacientes_b)
      ,sum(A.numero_pacientes) - sum(B.numero_pacientes_b) as diff
      ,b.cidade_b
    FROM


    (SELECT
        SUM(numero_pacientes) AS numero_pacientes_b, mm.municipio as cidade_b
    FROM
        cancer_mama.dados_municipios_mensal mm

    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(mm.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'

    group by cidade_b
    order by 2) B

    LEFT JOIN

    (SELECT
        count(distinct(pp.paciente)) AS numero_pacientes, cadastro_cidades.nome_uf, int(cadastro_cidades.id / 10) as cidade
    FROM
        cancer_mama.procedimentos_e_pacientes pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.primeiro_municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'
    group by nome_uf, cidade
    order by 3) A


    ON A.cidade = B.cidade_b

    group by b.cidade_b
    order by diff desc


""").show(1000)

+---------------------+-----------------------+----+--------+
|sum(numero_pacientes)|sum(numero_pacientes_b)|diff|cidade_b|
+---------------------+-----------------------+----+--------+
|                  458|                    442|  16|  314330|
|                  302|                    289|  13|  313130|
|                  121|                    114|   7|  314810|
|                  132|                    127|   5|  312610|
|                   67|                     63|   4|  313720|
|                  116|                    112|   4|  316470|
|                   54|                     51|   3|  310710|
|                   41|                     38|   3|  311420|
|                   10|                      7|   3|  311650|
|                  106|                    103|   3|  312980|
|                  219|                    216|   3|  315460|
|                    3|                      1|   2|  310370|
|                   25|                     23|   2|  310630|
|       

In [ ]:
spark.sql(f"""

select
   *
FROM
        cancer_mama.procedimentos_e_pacientes
where  paciente in
    (select paciente FROM
      (SELECT
            pp.paciente,
            count(distinct(cadastro_cidades.id)) as num_cidade_1,
            count(distinct( int(cadastro_cidades.id / 10) )) as num_cidade_2
        FROM
            cancer_mama.procedimentos_e_pacientes pp
        LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
        ON int(pp.municipio) = int(cadastro_cidades.id / 10)
        where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'
        group by pp.paciente
        order by 2 desc, 3 desc)
    where num_cidade_1 > 1) and data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'
""").show()

In [52]:
spark.sql(f"""
    SELECT
        data,
        primeiro_municipio as municipio,
        primeiro_estadiamento,
        SUM(custo) AS custo_estadiamento,
        COUNT(DISTINCT(paciente)) AS numero_pacientes,
        SUM(DISTINCT(obito)) AS obitos,
        SUM(DISTINCT(indicacao_obito)) AS obito_futuro,
        COUNT(1) AS numero_procedimentos
    FROM
        (SELECT * FROM cancer_mama.procedimentos_e_pacientes ORDER BY data)
    GROUP BY data, primeiro_municipio, primeiro_estadiamento
""").show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
spark.sql(f"""

  SELECT
        *
    FROM

  (SELECT
      c.*,
      p.data_primeiro_estadiamento,
      p.data_ultimo_estadiamento,
      p.primeiro_estadiamento,
      p.maior_estadiamento,
      p.ultimo_estadiamento,
      p.custo_total,
      p.primeiro_municipio,
      p.ultimo_municipio,
      p.indicacao_obito
  FROM (select pp.* from cancer_mama.procedimentos pp
        LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
        ON int(pp.municipio) = int(cadastro_cidades.id / 10)
       where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais') AS c
  RIGHT JOIN
       (select pp.* from cancer_mama.pacientes pp
          LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
        ON int(pp.primeiro_municipio) = int(cadastro_cidades.id / 10)
       where cadastro_cidades.nome_uf = 'Minas Gerais' )AS p
  ON c.paciente = p.paciente
  where data is not null)

""").createOrReplaceTempView("procedimentos_corrigido")

In [ ]:
spark.sql(f"""
    SELECT
        *
    FROM
        cancer_mama.procedimentos_e_pacientes pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais' and paciente in


    (SELECT paciente
    from
    (SELECT
        count(distinct(id)) cidades, paciente
    FROM
        procedimentos_corrigido pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.primeiro_municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'
    group by paciente
    order by 1 desc)
    --where cidades >1
    -- limit 1
    )

""").show()

+------+---------------+------------+------+-----+---------+--------------------------+------------------------+---------------------+------------------+-------------------+------------------+------------------+----------------+---------------+-------+----------------+-----+------------+
|  data|       paciente|estadiamento| custo|obito|municipio|data_primeiro_estadiamento|data_ultimo_estadiamento|primeiro_estadiamento|maior_estadiamento|ultimo_estadiamento|       custo_total|primeiro_municipio|ultimo_municipio|indicacao_obito|     id|            nome|id_uf|     nome_uf|
+------+---------------+------------+------+-----+---------+--------------------------+------------------------+---------------------+------------------+-------------------+------------------+------------------+----------------+---------------+-------+----------------+-----+------------+
|202501||{|{~|~{{{||           3|2378.9|    0|   313380|                    201902|                  202502|                    3

In [ ]:
spark.sql(f"""

select *
from cancer_mama.pacientes
where paciente in
(
SELECT paciente
    from
    (SELECT
        count(distinct(id)) cidades, paciente
    FROM
        procedimentos_corrigido pp
    LEFT JOIN ibge_silver.cadastro_municipios AS cadastro_cidades
    ON int(pp.municipio) = int(cadastro_cidades.id / 10)
    where data = '202501' and cadastro_cidades.nome_uf = 'Minas Gerais'
    group by paciente
    order by 1 desc)
    where cidades >1
    -- limit 1
    )
""").show()

+---------------+--------------------------+------------------------+--------------------+---------------------+-------------------+------------------+------------------+-----------+---------------+------------------+----------------+
|       paciente|data_primeiro_estadiamento|data_ultimo_estadiamento|numero_procedimentos|primeiro_estadiamento|ultimo_estadiamento|maior_estadiamento|menor_estadiamento|custo_total|indicacao_obito|primeiro_municipio|ultimo_municipio|
+---------------+--------------------------+------------------------+--------------------+---------------------+-------------------+------------------+------------------+-----------+---------------+------------------+----------------+
|{{{}{}{|                    202411|                  202501|                   4|                    2|                  2|                 2|                 2|    6143.25|              0|            312800|          312800|
|{{{|{|~}|                    202410|                

In [ ]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes
    FROM
        cancer_mama.dados_municipios_mensal
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          145963|
+----------------+



In [ ]:
spark.sql(f"""
    SELECT
        SUM(custo_estadiamento) AS custo_pacientes
    FROM
        cancer_mama.dados_estados_mensal
    where data = '202501'

""").show()

+-------------------+
|    custo_pacientes|
+-------------------+
|7.746116284999998E7|
+-------------------+



In [ ]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes
    FROM
        cancer_mama.dados_municipios_mensal
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          145963|
+----------------+



In [ ]:
spark.sql(f"""
    SELECT
        *
    FROM
        cancer_mama.dados_estados_mensal


""").show()

+--------+------+---------------------+------------------+----------------+-----------------+------+--------------+--------------------+-------------------+
|  estado|  data|primeiro_estadiamento|custo_estadiamento|numero_pacientes|numero_municipios|obitos|obitos_futuros|numero_procedimentos|numero_diagnosticos|
+--------+------+---------------------+------------------+----------------+-----------------+------+--------------+--------------------+-------------------+
|Rondônia|200801|                    1|1437.6999999999998|              13|                7|     0|             0|                  13|                 13|
|Rondônia|200801|                    2|17133.420000000002|              52|               19|     0|             2|                  53|                 52|
|Rondônia|200801|                    3|           50399.3|              70|               25|     0|             2|                  71|                 70|
|Rondônia|200801|                    4|          34021.97|

In [ ]:
spark.sql(f"""
    select sum(num_mun), count(1) from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM
        cancer_mama.procedimentos_e_pacientes
     where data = '202501'
     group by paciente)

     where num_mun > 1

""").show()

+------------+--------+
|sum(num_mun)|count(1)|
+------------+--------+
|        2409|     117|
+------------+--------+



In [ ]:
143671 - 145963

-2292

In [ ]:
2409 - 117

2292

In [ ]:
spark.sql(f"""
    select * from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM
        cancer_mama.procedimentos_e_pacientes
     where data = '202501'
     group by paciente)

     where num_mun > 1

""").show()

+-------+---------------+
|num_mun|       paciente|
+-------+---------------+
|   1792|               |
|    387|{{{{{{{{{{{{{{{|
|      2||{~{{{{|
|      2|}{||{{{{|
|      2|}{}{{{{|
|      2|}{|~~{{{|
|      2|{{{{|{|
|      2|{{{{}}}~{|
|      2|{{{{~~{~{|
|      2|{{{{~}{{~|
|      2|{{{{~{{|
|      2|{{{{~}||{}|
|      2|{{{{}|{{|
|      2|{{{{{|{|
|      2|{{{{|~{|
|      2|{{{{{}{~|
|      2|{{{{{}~|{|
|      2|{{{{}~}{|
|      2|{{}{{}~|
|      2|{{}{{}|
+-------+---------------+
only showing top 20 rows



In [ ]:
spark.sql(f"""
    select *, length(paciente) from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM

  (SELECT
      c.*,
      p.data_primeiro_estadiamento,
      p.data_ultimo_estadiamento,
      p.primeiro_estadiamento,
      p.maior_estadiamento,
      p.ultimo_estadiamento,
      p.custo_total,
      p.primeiro_municipio,
      p.ultimo_municipio,
      p.indicacao_obito
  FROM cancer_mama.pacientes AS p
  LEFT JOIN cancer_mama.procedimentos AS c
  ON c.paciente = p.paciente)

     where data = '202501'
     group by paciente)

     where num_mun > 1
     order by num_mun desc
     limit 2

""").show()

+-------+--------+----------------+
|num_mun|paciente|length(paciente)|
+-------+--------+----------------+
|   1792|        |               0|
+-------+--------+----------------+



In [ ]:
spark.sql(f"""
select * from

(    select *, length(paciente) from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM

  (SELECT
      c.*,
      p.data_primeiro_estadiamento,
      p.data_ultimo_estadiamento,
      p.primeiro_estadiamento,
      p.maior_estadiamento,
      p.ultimo_estadiamento,
      p.custo_total,
      p.primeiro_municipio,
      p.ultimo_municipio,
      p.indicacao_obito
  FROM cancer_mama.pacientes AS p
  LEFT JOIN cancer_mama.procedimentos AS c
  ON c.paciente = p.paciente)

     where data = '202501'
     group by paciente)

     where num_mun > 1
     order by num_mun desc
) a
left join cancer_mama.pacientes AS p
on a.paciente = p.paciente

""").show()

+-------+---------------+----------------+---------------+--------------------------+------------------------+--------------------+---------------------+-------------------+------------------+------------------+--------------------+---------------+------------------+----------------+
|num_mun|       paciente|length(paciente)|       paciente|data_primeiro_estadiamento|data_ultimo_estadiamento|numero_procedimentos|primeiro_estadiamento|ultimo_estadiamento|maior_estadiamento|menor_estadiamento|         custo_total|indicacao_obito|primeiro_municipio|ultimo_municipio|
+-------+---------------+----------------+---------------+--------------------------+------------------------+--------------------+---------------------+-------------------+------------------+------------------+--------------------+---------------+------------------+----------------+
|   1792|               |               0|               |                    202404|                  202502|               88593|              